In [2]:
import socket
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import json

# function to receive all data
def recv_all(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf:
            return None
        buf += newbuf
        count -= len(newbuf)
    return buf

# initialize the server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('192.168.0.84', 8085))  # replace 'localhost' with your IP address
server_socket.listen(1)

# load the YOLO model
net = cv2.dnn.readNet('/home/davorin/Documents/OP/yolov3.weights', 'yolov3.cfg')

while True:
    print("Waiting for a connection...")
    client_socket, addr = server_socket.accept()
    print(f"Accepted connection from {addr}")

    # receive the image data
    length_prefix = recv_all(client_socket, 4)  # assuming that the length is sent as 4 bytes (a 32-bit int)
    if length_prefix is None:
        print('Length prefix not received')
    else:
        length = int.from_bytes(length_prefix, 'little')  # convert bytes to int
        data = recv_all(client_socket, length)  # receive the rest of the data
        if data is None:
            print('Image not received')
        else:
            print("Loading classes")
            with open("coco.names", "r") as f:
                classes = [line.strip() for line in f.readlines()]
            print("Classes loaded")

            # convert the bytes to an image
            image_stream = BytesIO(data)
            image = Image.open(image_stream)
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            # use the YOLO model to perform object detection
            print("Image is being processed")
            height, width = image.shape[:2]
            blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(net.getUnconnectedOutLayersNames())
            print("Inference running")

            # process the results
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            print("Coordinates are here:")

            # apply non-maxima suppression to
            # apply non-maxima suppression to get the most relevant detections
            indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            # create a list to store all the detection results
            results = []
            if len(indices) > 0:
                for i in indices:
                    index = i[0]
                    box = boxes[index]
                    x, y, w, h = box[0], box[1], box[2], box[3]
                    label = str(classes[class_ids[index]])
                    result = {'label': label, 'x': x, 'y': y, 'w': w, 'h': h}
                    results.append(result)
            print(results)

            # send the detection results back to the client
            response = json.dumps(results).encode()
            response_length = len(response)
            client_socket.sendall(response_length.to_bytes(4, 'little'))
            client_socket.sendall(response)

    client_socket.close()


Waiting for a connection...
Accepted connection from ('192.168.0.215', 64331)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64332)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64333)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64349)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64350)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64351)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64409)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64410)
Image not received
Waiting for a connection...
Accepted connection from ('192.168.0.215', 64411)
Image not received
Waiting for a connection...


KeyboardInterrupt: 